In [1]:
import uuid

import pandas as pd

In [2]:
ini_df = pd.read_csv('data/anketa_ini.csv')

# Предварительная подготовка таблиц #

In [3]:
display(ini_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 14 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   Отметка времени                                                   320 non-null    object
 1   Профиль телеграмм (в формате @username)                           321 non-null    object
 2   Часовой пояс                                                      322 non-null    object
 3   Стек технологий                                                   321 non-null    object
 4   По какой специальности SF хотите заявиться?                       322 non-null    object
 5   Роль                                                              322 non-null    object
 6   В какой роли видите себя в проекте?                               322 non-null    object
 7   Сколько часов в неделю готовы уделять проект

None

#### Переименование признаков для размещения таблиц в БД ####

In [4]:
old_cols = [
    'Отметка времени',
    'Профиль телеграмм (в формате @username)',
    'Часовой пояс',
    'Стек технологий',
    'По какой специальности SF хотите заявиться?',
    'Роль',
    'В какой роли видите себя в проекте?',
    'Сколько часов в неделю готовы уделять проекту?',
    'Какие другие курсы закончили или находитесь в процессе обучения?',
    'Как долго учитесь на курсах?',
    'Notes',
    'ЯП',
    'Вступление в чат практики.',
    'Выбыл'
]
new_cols = [
    'date',
    'tg_id',
    'timezone',
    'stack',
    'occupation',
    'role',
    'project_role',
    'weekload',
    'course',
    'course_time',
    'notes',
    'pl',
    'chat',
    'exited'
]

def map_names(old_list: list, new_list: list) -> dict:
    dict_ = dict()
    if len(old_list) == len(new_list):
        for old_name, new_name in zip(old_list, new_list):
            dict_[old_name] = new_name
    return dict_

ini_df.rename(columns=map_names(old_cols, new_cols), inplace=True)
ini_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          320 non-null    object
 1   tg_id         321 non-null    object
 2   timezone      322 non-null    object
 3   stack         321 non-null    object
 4   occupation    322 non-null    object
 5   role          322 non-null    object
 6   project_role  322 non-null    object
 7   weekload      322 non-null    object
 8   course        322 non-null    object
 9   course_time   322 non-null    object
 10  notes         99 non-null     object
 11  pl            205 non-null    object
 12  chat          322 non-null    object
 13  exited        63 non-null     object
dtypes: object(14)
memory usage: 35.3+ KB


#### Контроль пустых мест ####

In [5]:
display(ini_df.isna().sum(axis='index'))

date              2
tg_id             1
timezone          0
stack             1
occupation        0
role              0
project_role      0
weekload          0
course            0
course_time       0
notes           223
pl              117
chat              0
exited          259
dtype: int64

In [6]:
ini_df.tg_id.nunique()

321

#### Выделение идентификаторов пользователей в отдельную таблицу **`private_table.csv`** ####

In [7]:
ini_df['uuid'] = ''
ini_df.uuid = ini_df.uuid.transform(lambda x: str(uuid.uuid4()))
private_table_df = ini_df[['uuid', 'tg_id']]
private_table_df.to_csv('data/private_table.csv')

#### Удаление из основной таблицы telegram-логинов (остаются только **`uuid`**) ####

In [8]:
table_df = ini_df.drop('tg_id', axis='columns')
table_df.to_csv('data/table.csv')

In [9]:
table_df.describe(include='object')

,date,timezone,stack,occupation,role,project_role,weekload,course,course_time,notes,pl,chat,exited,uuid
count,320,322,321,322,322,322,322,322,322,99,205,322,63,322
unique,320,7,260,84,13,251,3,309,111,18,26,3,1,322
top,29.06.2023 20:42:20,GMT+3 (Москва),Python,Тестирование,Тестировщик,Тестировщик,20+ часов,Skillbox,Заканчиваю,manual,Python,Да,Выбыл,e51c7642-2e5b-4734-a9b1-012fefecf86c
freq,1,246,21,86,93,17,135,4,65,47,39,243,63,1


In [10]:
table_df.head()

,date,timezone,stack,occupation,role,project_role,weekload,course,course_time,notes,pl,chat,exited,uuid
0,29.06.2023 20:42:20,GMT+3 (Москва),"Vue.js(Vue 3), Vuex, Vue Router, JavaScript(ES...","Программирование, Веб - разработка, Backend - ...",Frontend-разработчик,Frontend-разработчик,готов работать 25/8,web-разработчик на python,Нахожусь в конце обучения,NaN,JS,Да,Выбыл,e51c7642-2e5b-4734-a9b1-012fefecf86c
1,29.06.2023 21:35:28,GMT+3 (Москва),"python (pandas. numpy, requests, bs4 и т.д.), ...","Data Science, Аналитика данных, Программирован...",Аналитик,"DS, сбор и анализ данных, python-разработка",20+ часов,SkillFactory DS,Нахожусь в середине обучения,DS,Python,Да,NaN,fc96e578-a226-44a9-bb74-d3191035d5a9
2,29.06.2023 21:36:43,GMT+3 (Москва),"UI\UX дизайн, графический дизайн","Разработка приложений, Разработка игр, Дизайн",Дизайнер,Дизайнер,20+ часов,UI\UX дизайн,Нахожусь в конце обучения,NaN,дизайн,Да,NaN,4c3ea95d-3999-4671-94c1-1473f21c97b4
3,29.06.2023 21:37:21,GMT+3 (Москва),ui/ux; graphic;Figma;Тильда;Логодизайнер,"Дизайн, Создание сайтов",Дизайнер,графический дизайнер,10+ часов,Поток;Contented;UNID,Закончил(а) обучение,NaN,NaN,Нет,NaN,ee67718d-49f8-4071-b40a-d5a99ba336a9
4,29.06.2023 21:38:50,GMT+3 (Москва),c# Backend,"Программирование, Разработка игр, Веб - разраб...",Backend разработчик,Программист-разработчик backend,готов работать 25/8,"SkillBox, Synergy, Ulearn... (в основном везде...",Закончил(а) обучение,NaN,C#,Да,NaN,3b85d8dd-1c29-4609-8e48-6413446c24a5


In [11]:
table_df.pl.unique()

array(['JS', 'Python', 'дизайн', nan, 'C#', 'Java', 'Golang', 'QAM',
       'QAP', 'DevOps', 'Android', 'PHP', 'С++', 'Andoroid', 'SQL',
       'дизайнер', 'QAJA', 'QAP + Lead', 'QAJA + Lead', 'C++',
       'QAM + Lead', 'Flutter Dart', 'iOS', 'PM', 'JS + Lead', 'QAM+Lead',
       'QA'], dtype=object)